<center>
<h1><br\></h1>
<h1>INF582: INTRODUCTION TO TEXT MINING AND NLP</h1>
</center>
<center>
<h2>Lab Session 1: TF-IDF</h2>
<h4>Lecture: Prof. Michalis Vazirgiannis<br>
Lab: Dr Guokan Shang and Hadi Abdine <br> contact: hadi.abdine@polytechnique.edu</h4>
<h5>Friday, January 12, 2024</h5>
<h4><b>Student Name:</b> </h4>
<br>
</center>

<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit to Moodle a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing a your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>January 19, 2024 08:29 AM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).
</p>
<hr style="border:5px solid gray"> </hr>

<h3><b>1. Introduction</b></h2>
<p style="text-align: justify;">
Documents are traditionally represented with the vector space model, also known as the bag-of-words representation [<a href='https://nlp.stanford.edu/IR-book/' >Manning et al.</a>]. With this approach, each document di from the collection D = {d1 . . . dm } of size m is associated with an n-dimensional feature vector, where n is the number of unique terms in the preprocessed collection. The set of unique terms T = {t1 . . . tn } is called the vocabulary.
Term Frequency-Inverse Document Frequency (TF-IDF), is a method used to compute the coordinates of a document in the vector space.
</p>

<h3><b>2. Learning Objective</b></h2>
<p style="text-align: justify;">
In this lab, you will learn how to compute the TF-IDF representation and use it in 3 different tasks:
<ul>
<li>computing the cosine similarity between documents</li>
<li>executing a query against a collection of documents using the inverted index,</li>
<li>performing supervised document classification.</li>
</ul>
</p>

<h3><b>3. Computing TF-IDF</b></h2>
<p style="text-align: justify;">
The assumption is that the importance of a word to a document increases when its frequency increases. However, considering the frequency as the only factor to judge the importance of a word would result in giving greater weight to commonly used terms such as stopwords, which could be misleading in many tasks. TF-IDF mitigates this problem by introducing a factor that diminishes the weight of words that occur frequently in other documents in the same collection. The TF-IDF weight computation is based on the product of two separate factors, namely the Term Frequency (TF) and the Inverse Document Frequency (IDF). More specifically:

$$ tf\text{-}idf(t,d,\mathcal{D}) = tf(t,d) \times idf(t, \mathcal{D}) $$

There are many ways to determine tf and idf . In our case, $tf (t, d)$ is the number of times term $t$ appears in document $d$, and $idf (t, D) = ln (\frac{m}{1+df (t)}) + 1$, with $df (t)$ the number of documents in the collection $D$ that contains $t$. We can notice that the weight of a term in a document increases when its frequency increases in this document (first factor), and decreases when the number of documents in the collection containing this term increases (second factor). M ∈ $R^{m×n}$ is the TF-IDF matrix of $D$, where $M_{ij}$ is the TF-IDF weight of the $jth$ word in the $ith$ document.

</p>




### Importing libraries and downloading the data:

In [1]:
import nltk
import numpy as np
import time

from nltk import word_tokenize
nltk.download('punkt')

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
import warnings
warnings.filterwarnings("ignore")

import urllib

urllib.request.urlretrieve("https://onedrive.live.com/download?cid=AE69638675180117&resid=AE69638675180117%2152573&authkey=AFz5kPjESCHRl3s", 'webkb-train-stemmed.txt')
urllib.request.urlretrieve("https://onedrive.live.com/download?cid=AE69638675180117&resid=AE69638675180117%2152576&authkey=ACypGA77xWokzQ8", 'webkb-test-stemmed.txt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


('webkb-test-stemmed.txt', <http.client.HTTPMessage at 0x2cf747247c0>)


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 1: </b><br>
The next cell implements a <i>tfidf</i> class. When initialized, a <i>tfidf</i> object takes as input a collection of documents and computes the <i>IDF</i> of words in this collection. You can provide a list of stopwords to be ignored. Two methods are defined in this class: <i>tf</i> that returns the frequency of unique words in a document and transform that returns the <i>TF-IDF</i> matrix of the collection. Fill in the gaps in the different functions of <i>tfidf</i> class.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [28]:
documents = ["euler is the father of graph theory",
             "graph theory studies the properties of graphs",
             "bioinformatics studies the application of efficient algorithms in the biological field"]

class tfidf(object):
    def __init__(self, collection, stop_words=[]):
        '''collection is a list of strings'''
        self.word2ind = {} # map each word in the collection to a unique index
        self.ind2word = {}
        self.word2idf = {} # map each word to its idf
        self.collection = collection
        self.documents = [document.split() for document in collection]

        all_words = [] # fill it, list of all words in the collection
        for document in self.documents:
            all_words.extend(document)
        self.unique_words = list(set(all_words)) # fill it, list of unique words in the collection
        self.unique_words = [word for word in self.unique_words if word not in stop_words] # remove stopwords
        self.count_unique_words = len(self.unique_words)
        self.word2ind = dict(zip(self.unique_words,range(self.count_unique_words)))
        self.ind2word = {v:k for k,v in self.word2ind.items()}
        self.count_documents = len(collection)

        # compute the idf of unqiue words in the collection
        for word in self.word2ind.keys():
            count = 0 # fill it, number of documents that contains word
            for document in self.documents:
                if word in document:
                    count += 1
            idf = np.log(self.count_documents / (1 + count)) + 1 # fill it
            self.word2idf[word] = idf

    def getWordFromInd(self, ind):
        return self.ind2word[ind]

    def getListWords(self):
        return self.unique_words

    def tf(self, document):
        '''
        return the frequency of each unique word in document.
        document is a list of strings
        '''
        word2frequency = {}
        for word in document:
            word2frequency[word] = word2frequency.get(word, 0) + 1 # increment, creating key if it doesn't already exist
        return word2frequency

    def transform(self, collection):
        documents = [document.split() for document in collection] # tokenize documents in the collection
        tfidf_mat = np.zeros((len(collection), self.count_unique_words)) # fill it, intialize tfidf matrix with zeros
        # compute tfidf
        for ind, document in enumerate(documents):
            word2frequency = self.tf(document)
            for word in word2frequency.keys():
                if word in self.word2ind:
                    tfidf_mat[ind, self.word2ind[word]] = self.word2idf[word] * word2frequency[word] # fill it, tfidf
        return tfidf_mat


<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 1 (5 points): </b><br>
In the above script we provide a small collection documents. In order to validate your functions, compute manually the TF-IDF of the word <b>’the’</b> in the last document and check if you get the same result as with your code.
<hr style="border:10px solid red"> </hr>
</font></h4>

In [29]:
#tfidf of 'the' in last document
t = tfidf(documents)
tfidf_mat = t.transform(documents)
index = t.word2ind['the']
print(tfidf_mat[-1][index])

1.4246358550964382


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 1: </b><br>
Your answer here.
<hr style="border:10px solid green"> </hr>
</font></h4>

<h3><b>4. Cosine Similarity</b></h2>
<p style="text-align: justify;">
In this section we will compute the cosine similarity of two documents using the TF-IDF representation. The similarity concept is crucial in search engines as well as in text mining applications.

$$
\begin{equation}
\mathrm{similarity}(v_1, v_2) = \frac{v_1 \cdot v_2}{\|v_1\| \times \|v_2\|}
\end{equation}
$$

Normally, the cosine similarity ranges from -1 to +1. However, in our case all the entries of the TF-IDF features are positive, thus it ranges from 0 to 1. The cosine similarity matrix is a square matrix representing the similarity between all pairs of documents in a collection. In other words, if S is the similarity matrix, then $S_{ij} = similarity(v_i , v_j)$ where $v_i$ is the TF-IDF vector of the ith document and $v_j$ is the TF-IDF vector of the jth document.

</p>


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 2: </b><br>
Complete the two functions $\texttt{cosine_similarity()}$ and $\texttt{cosine_similarity_matrix()}$ in the next cell.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [30]:
documents = ["i like that music",
	         "this song appeals to me",
	         "i like graph theory",
	         "euler is the father of graph theory",
           "graph theory studies the properties of graphs",
           "the quick brown fox jumps over the lazy dog",
           "the quick fox jumps over the brown lazy dog"]

def cosine_similarity(v1, v2):
    '''returns the cosine similarity of 2 vectors'''
    v1 = np.array(v1)
    v2 = np.array(v2)
    numerator = v1.dot(v2) # fill it
    denominator = np.linalg.norm(v1) * np.linalg.norm(v2) # fill it
    return numerator / denominator

def cosine_similarity_matrix(mat):
    '''
    returns the cosine similarity matrix
    the ith row in mat represents the ith vector
    '''
    similarity_matrix = np.zeros((mat.shape[0], mat.shape[0])) # fill it
    for i in range(mat.shape[0]):
        for j in range(mat.shape[0]):
            similarity_matrix[i][j] = cosine_similarity(mat[i], mat[j]) # fill it
    return similarity_matrix


<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 2 (2 points): </b><br>
Run the code of the next cell and examine its output. What do you observe about the similarity matrix? What can we do to speed-up the computation?
<hr style="border:10px solid red"> </hr>
</font></h4>

In [31]:
t = tfidf(documents)
tfidf_matrix = t.transform(documents)
similarity_matrix = cosine_similarity_matrix(tfidf_matrix)
print(similarity_matrix)

[[1.         0.         0.4845028  0.         0.         0.
  0.        ]
 [0.         1.         0.         0.         0.         0.
  0.        ]
 [0.4845028  0.         1.         0.28294469 0.28294469 0.
  0.        ]
 [0.         0.         0.28294469 1.         0.39794976 0.12752163
  0.12752163]
 [0.         0.         0.28294469 0.39794976 1.         0.12752163
  0.12752163]
 [0.         0.         0.         0.12752163 0.12752163 1.
  1.        ]
 [0.         0.         0.         0.12752163 0.12752163 1.
  1.        ]]


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 2: </b><br>
The similarity matrix is symmetric and the diagonal is 1. We can speed-up the computation by computing only the upper triangular part of the matrix and then copy it to the lower triangular part.
<hr style="border:10px solid green"> </hr>
</font></h4>

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 3 (3 points): </b><br>
Notice the similarity between first two documents and between last two documents and state two drawbacks of the bag-of-words representation.
<hr style="border:10px solid red"> </hr>
</font></h4>

In [32]:
similarity_1_2 = similarity_matrix[0][1]
similarity_last = similarity_matrix[-1][-2]

print('The similarity between the first and second documents is: ', similarity_1_2)
print('The similarity between the two last documents is: ', similarity_last)

The similarity between the first and second documents is:  0.0
The similarity between the two last documents is:  1.0


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 3: </b><br>
Although the first and second documents deal with the same subject, music, the similarity between them is 0. This is because the bag-of-words representation does not take into account the context in which the words appear. Synonyms, variations in word form or changes in word order are not taken into account, here in the first document with "music" and in the second with "song", which are treated as different.
Also, based on the similarity of the two last document, we can say that as the bags-of-words treats each document like an unordered set of words, disregarding the order and structure of the words. This can lead to a loss of semantic meaning.
<hr style="border:10px solid green"> </hr>
</font></h4>

<h3><b>5. Inverted Index</b></h2>
<p style="text-align: justify;">
Typically, search engines execute queries against a huge document collection.
To execute a query we can simply calculate a similarity measure between the query and all the documents in the collection.<br>
However, doing so is very inefficient. Instead, it is possible to eliminate a lot of documents from the candidate set by using the inverted index.
The inverted index consists of a mapping from words to documents.
Each unique word in the vocabulary is mapped to a list containing the documents in which the word appears and the position of the word in these documents. Stemming, which is reducing tokens to a root form, could be useful in this task. For example a stemming algorithm would reduce <i>computing</i>, <i>computers</i> and <i>computation</i> to <i>comput</i> and thus identifying them as one unique token instead of three different ones. In our code we consider two dictionaries to perform the mapping. In one of the dictionaries the words are stemmed, and in the other they are not.
Instead of ranking all the documents in the collection, the inverted index allows us to rank only the relevant documents.

In our code we provide several functions:
<ul>
<li><i>at_least_one_unigram()</i>, returns documents containing at least one query word.
<li> <i>all_unigrams()</i>, returns documents containing all query words.
<li> <i>ngrams()</i>, returns documents containing all query words in the same order as in the query.
</ul>

Then, we test all these functionalities using a small corpus containing a limited number of documents. Finally, we execute a query the naive way (examining all documents) and using the inverted index and we compare the execution time.
</p>


In [33]:
stemmer = nltk.stem.PorterStemmer()

# = = = = = = = = = = = =

def clean_tokenize(doc):
    words = word_tokenize(doc.lower())
    return words

def index_one_doc(doc,to_stem):
    '''
    creates dict (tok,positions) for each tok in the document (as term list)
    '''
    tokpos = dict()
    for t_idx,tok in enumerate(doc):
        if to_stem:
           tok = stemmer.stem(tok)
        if tok in tokpos:
            tokpos[tok].append(t_idx)
        else:
            tokpos[tok] = [t_idx]
    return tokpos


# = = = = = = = = = = = =

docs = ['The quick brown fox jumps over the lazy dog',
        'The brown quick fox jumps over the lazy dog',
        'Luke is the mechanical and electrical engineer of the new group',
        'Instead of buying a new engine, buy a new car',
        'An engineer may design car engines of all sorts',
        'Engineers use logic, but not necessarily imagination',
        'Logic will take you from A to Z, imagination will take you everywhere.',
        'Continuous effort, not strength or intelligence, is the key to \
        unlocking our potential. And curiosity.',
        'It’s OK to have your eggs in one basket as long as you control what \
        happens to that basket.'
        ]

cleaned_docs = []
for doc in docs:
    to_app = clean_tokenize(doc)
    cleaned_docs.append(to_app)

# = = = = = = = = = = = = = = =

index_one_doc(cleaned_docs[3],to_stem=True)

index_one_doc(cleaned_docs[3],to_stem=False)

'''
- queries are not case sensitive
- we are indexing punctuation marks
- we index stopwords and should keep stopwords in the queries (gives more expressivity)
'''

inverted_index = dict()
inverted_index_stem = dict()

for d_idx,doc in enumerate(cleaned_docs):

    poslists_s = index_one_doc(doc,to_stem=True) # get positions of each token in the doc
    for tok,poslist_s in poslists_s.items():
        if tok in inverted_index_stem:
            inverted_index_stem[tok][d_idx] = poslist_s # update
        else:
            inverted_index_stem[tok] = dict()
            inverted_index_stem[tok][d_idx] = poslist_s # initialize

    poslists = index_one_doc(doc,to_stem=False)
    for tok, poslist in poslists.items():
        if tok in inverted_index:
            inverted_index[tok][d_idx] = poslist
        else:
            inverted_index[tok] = dict()
            inverted_index[tok][d_idx] = poslist


# = = = = = = = = = = = = = = = = =

def at_least_one_unigram(query,inverted_index):
    '''
    returns the indexes of the docs containing *at least one* query unigrams
    the query is a list of unigrams
    '''

    to_return = []
    for unigram in query:
        if unigram in inverted_index:
            to_return.extend(list(inverted_index[unigram].keys()))
    return list(set(to_return))

def all_unigrams(query,inverted_index):
    '''
    returns the indexes of the docs containing *all* query unigrams
    the query is a list of unigrams
    '''

    to_return = []
    for unigram in query:
        if unigram in inverted_index:
            to_return.append(set(list(inverted_index[unigram].keys())))
        else:
            to_return.append(set())
            break
    to_return = to_return[0].intersection(*to_return)
    return list(to_return)

def ngrams(query,inverted_index):
    '''
    returns the indexes of the docs containing all unigrams in same order as the query
    the query is a list of unigrams
    '''
    candidate_docs = all_unigrams(query,inverted_index)

    to_return = []
    for doc in candidate_docs:
        poslists = []
        for unigram in query:
            to_append = inverted_index[unigram][doc]
            if isinstance(to_append, int):
                poslists.append([to_append])
            else:
                poslists.append(to_append)
        # test whether the query words are consecutive
        poslists_sub = [[elt-idx for elt in poslist] for idx,poslist in enumerate(poslists)]
        if set(poslists_sub[0]).intersection(*poslists_sub):
            to_return.append(doc)
    return to_return

## Queries:

In [35]:
query = ['engine','car']
print('Query: {}'.format(' '.join(query)))
print('-----------------------------------------------')

docs_index = at_least_one_unigram(query,inverted_index)
print('docs containing at least one word in the query:')
for el in docs_index:
    print('* {}'.format(docs[el]))

print()

docs_index = all_unigrams(query,inverted_index)
print('docs containing all the words in the query:')
for el in docs_index:
    print('* {}'.format(docs[el]))

print()

query_stemmed = [stemmer.stem(elt) for elt in query]
docs_index = at_least_one_unigram(query_stemmed,inverted_index_stem)
print('docs (stemmed) containing at least one word in the query (stemmed):')
for el in docs_index:
    print('* {}'.format(docs[el]))

print()

docs_index = all_unigrams(query_stemmed,inverted_index_stem)
print('docs (stemmed) containing all the words in the query (stemmed):')
for el in docs_index:
    print('* {}'.format(docs[el]))

print()

docs_index = ngrams(query,inverted_index)
print('docs containing engine all the words in the query in the same order:')
for el in docs_index:
    print('* {}'.format(docs[el]))

print('********************************************************')
print()

tf_idf = tfidf(docs)
query = ['new', 'car']
tf_idf_query = tf_idf.transform([(' ').join(query)])

Query: engine car
-----------------------------------------------
docs containing at least one word in the query:
* Instead of buying a new engine, buy a new car
* An engineer may design car engines of all sorts

docs containing all the words in the query:
* Instead of buying a new engine, buy a new car

docs (stemmed) containing at least one word in the query (stemmed):
* Luke is the mechanical and electrical engineer of the new group
* Instead of buying a new engine, buy a new car
* An engineer may design car engines of all sorts
* Engineers use logic, but not necessarily imagination

docs (stemmed) containing all the words in the query (stemmed):
* Instead of buying a new engine, buy a new car
* An engineer may design car engines of all sorts

docs containing engine all the words in the query in the same order:
********************************************************




<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 3: </b><br>
Complete the code in the next cell.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [37]:
############ query over all documents
print('Query ({}) over all documents'.format((' ').join(query)))
print('-----------------------------------------------')
time1 = time.time()
tf_idf_collection = tf_idf.transform(docs) # fill it
size = tf_idf_collection.shape[0]
scores = [cosine_similarity(tf_idf_query, tf_idf_collection[i]) for i in range(size)] # fill it, list containing the cosine similarities of the query against all documents

time2 = time.time()
print("Top similar document: {}".format(docs[np.argmax(scores)]))
print("Found in {} ms".format((time2 - time1)*1000))

print('')

############ query over candidate documents using inverted index
print('Query ({}) over candidates containing at least one of the words'.format((' ').join(query)))
print('-----------------------------------------------')
time1 = time.time()
candidate_docs_index = at_least_one_unigram(query,inverted_index) # fill it
candidate_docs = [docs[el] for el in candidate_docs_index]
tf_idf_collection = tf_idf.transform(candidate_docs)
size = tf_idf_collection.shape[0]
scores = [cosine_similarity(tf_idf_query, tf_idf_collection[i]) for i in range(size)] # fill it, list containing the cosine similarities of the query against candidate documents
time2 = time.time()
print("Top similar document: {}".format(candidate_docs[np.argmax(scores)]))
print("Found in {} ms".format((time2 - time1)*1000))

Query (new car) over all documents
-----------------------------------------------
Top similar document: Instead of buying a new engine, buy a new car
Found in 0.0 ms

Query (new car) over candidates containing at least one of the words
-----------------------------------------------
Top similar document: Instead of buying a new engine, buy a new car
Found in 0.0 ms


<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 4 (2.5 points): </b><br>
Examine and interpret the output of the previous cell.
<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 4: </b><br>
As result we get the same document if we look into all the documents or the documents containing at least one query word, and this is logical because the score of a document that doesn't contains any word of the query is small that the score of a document that contains at least one word of the query as the TF-IDF is computed based on the frequency of the words in the document. So, looking into all the documents or the documents containing at least one query word gives the same result.
<hr style="border:10px solid green"> </hr>
</font></h4>

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 5 (2.5 points): </b><br>
If we execute a query the naive way, and then we execute it against the documents containing all the words in the query, is it possible to get different results? Motivate your answer
<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 5: </b><br>
Yes, it is possible to get a different result because if the list of documents containing all the words in the query is empty, the result will be empty, while the naive way will possibly return some documents since some documents may contain some of the words in the query. 
<hr style="border:10px solid green"> </hr>
</font></h4>

<h3><b>6. Supervised Classification</b></h2>
<p style="text-align: justify;">
In this section we will use the TF-IDF features to perform a supervised classification task. We will work with the WebKB dataset. It features academic webpages belonging to four different categories: (1) project, (2) course, (3) faculty, and (4) students, and contains 2,803 documents for training and 1,396 for testing. Documents have already been preprocessed with stopword removal and Porter stemming. The code that you will work with implements the following steps:

<ul>
<li>data loading,
<li>computation of TF-IDF features for the training set,
<li> computation of features for the test set. Note that the documents in the test set are represented in the space made of the unique terms in the training set only (words in the testing set absent from the training set are disregarded).
<li>classifier training/testing. Naive Bayes classifier [<a href='https://www.cs.cmu.edu/~knigam/papers/multinomial-aaaiws98.pdf'>McCallum and Nigam, 1998</a>], Multinomial Logistic Regression [<a href='https://www.learningtheory.org/colt2000/papers/CollinsSchapireSinger.pdf'>Collins et al., 2002</a>], Ran- dom Forest Classifier [1] and linear kernel SVM [<a href='https://link.springer.com/article/10.1007/BF00994018'>Cortes and Vapnik 1995</a>, <a href='https://www.researchgate.net/publication/28351286_Text_Categorization_with_Support_Vector_Machines'>Joachims 1998</a>] are compared,
<li>get the most/least important words per class.
</ul>

Then, we test all these functionalities using a small corpus containing a limited number of documents. Finally, we execute a query the naive way (examining all documents) and using the inverted index and we compare the execution time.
</p>



<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 4: </b><br>
Complete the code in the next cell.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [43]:
def print_top10(feature_names, clf, class_labels):
    """Prints features with the highest coefficient values, per class"""
    # coef stores the weights of each feature (in unique term), for each class
    for i, class_label in enumerate(class_labels):
        top10 = np.argsort(clf.coef_[i])[-10:]
        print("%s: %s" % (class_label," ".join(feature_names[j] for j in top10)))

def print_bot10(feature_names, clf, class_labels):
    """Prints features with the lowest coefficient values, per class"""
    for i, class_label in enumerate(class_labels):
        bot10 = np.argsort(clf.coef_[i])[0:9]
        print("%s: %s" % (class_label," ".join(feature_names[j] for j in bot10)))

def prepare_data(path):
    with open(path, 'r') as f:
        s = f.read()
    examples = s.split('\n') # split to samples
    examples = examples[:-1] # last element is empty
    documents = []
    labels = []
    for el in examples:
        example = el.split('\t') # separate document from label
        documents.append(example[1])
        labels.append(example[0])
    return documents, labels

path_train = './webkb-train-stemmed.txt' # path to train data
path_test = './webkb-test-stemmed.txt' # path to test data
train_documents, train_labels = prepare_data(path_train)
test_documents, test_labels = prepare_data(path_test)

categories = set(train_labels) # get unique categoris
category2ind = dict(zip(categories,range(len(categories)))) # map each category to index
ind2category = {v:k for k,v in category2ind.items()} # map index to category

train_labels_index = [category2ind[cat] for cat in train_labels] # replace labels by their indexes
test_labels_index = [category2ind[cat] for cat in test_labels] # replace labels by their indexes

t = tfidf(train_documents) # fill it, tfidf object
tfidf_train = t.transform(train_documents) # fill it, get the tfidf training matrix
tfidf_test = t.transform(test_documents) # fill it, get the tfidf text matrix

lr = LogisticRegression(multi_class='auto', solver='lbfgs', max_iter=200)
rf = RandomForestClassifier(n_estimators=20)
nb = MultinomialNB()
svm = LinearSVC(max_iter=2000)

Classifiers = {'Logisitc Regression': lr, 'Random forest': rf, 'Naive Bayes': nb, 'Linear SVM': svm}

for classifier in Classifiers.keys():
    Classifiers[classifier].fit(tfidf_train, train_labels_index) # train each classifier
    predicted = Classifiers[classifier].predict(tfidf_test) # perform prediction
    accuracy = 0 # fill it, compute accuracy
    for i in range(len(predicted)):
        if predicted[i] == test_labels_index[i]:
            accuracy += 1
    accuracy /= len(predicted)
    print('{} accuracy: {}'.format(classifier, accuracy))

predicted = np.zeros((len(test_labels_index))) + 3
print(np.mean(predicted == np.array(test_labels_index)))
print('')

# choose one classfier and print top 10 and bottom 10 words per class
feature_names = t.ind2word  # fill it
classifier = lr # fill it
print('Top 10')
print_top10(feature_names, classifier, categories)
print('Bottom 10')
print_bot10(feature_names, classifier, categories)

Logisitc Regression accuracy: 0.8810888252148997
Random forest accuracy: 0.8481375358166189
Naive Bayes accuracy: 0.8173352435530086
Linear SVM accuracy: 0.8474212034383954
0.2679083094555874

Top 10
student: address wisc graduat interest home zhang advisor work construct resum
course: hui grade structur materi data assign instructor comp syllabu fall
project: softwar faculti perform project peopl high lab hybrid request group
faculty: ufl cours perman recent cpsc associ teach henri fax professor
Bottom 10
student: professor henri perman faculti group fall hybrid comp fax
course: research cours interest vision berkelei cpsc person pictur graphic
project: interest home address fall email offic fax professor scienc
faculty: graduat lab student construct advisor homework syllabu move resum


<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 6 (5 points): </b><br>
Examine the accuracy of different classifiers, examine the most/least important words and comment the results.
<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 6: </b><br>
The accuracy of the different classifiers is pretty similar around 0.80 even if the Naive Bayes classifier is the least accurate. The most accurate is the Logistic Regression classifier, the random forest classifier and the Linear SVM are in the middle.

The most important words seem to be related to general academic or professional topics, such as "address," "research," "project," and "faculty." These words are likely indicative of the content or context of the data being classified.
The least important words appear to be a mix of specific terms and possibly less informative words, such as "professor," "fall," "hybrid," and "fax." It's important to note that without context, it's challenging to interpret the significance of these words accurately.

<hr style="border:10px solid green"> </hr>
</font></h4>